<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/07_Pandas_Basico.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🐼 Pandas Básico

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🟡 Intermedio  
**Duración estimada:** 75 minutos  
**Requisitos:** Haber completado el [Notebook 06 — Manejo de Archivos CSV y Excel](06_Manejo_de_Archivos_CSV_y_Excel.ipynb)

---

En este notebook vas a:
- Entender qué es Pandas y por qué es la herramienta #1 para datos en Python
- Crear y manipular DataFrames (las tablas de Pandas)
- Leer datos de CSV y Excel en una sola línea
- Filtrar, ordenar, agrupar y transformar datos
- Crear columnas calculadas y hacer análisis exploratorio
- Exportar resultados a CSV y Excel

> 💡 **Pandas** es a Python lo que Excel es a oficina — pero con superpoderes. Todo lo que hicimos en el notebook anterior con 20 líneas, aquí lo haremos en 2-3.


---

## 0. Preparación: generar datos de ejemplo

Ejecuta esta celda para crear los mismos datos del notebook anterior:


In [ ]:
import csv, os, random, openpyxl
from datetime import datetime, timedelta
from openpyxl.styles import Font, PatternFill

os.makedirs("datos", exist_ok=True)
random.seed(42)

sucursales = ["Centro", "Tres Ríos", "Plaza Fiesta", "Forum", "Galerías",
              "Mazatlán Centro", "Mazatlán Dorado", "Los Mochis", "Guasave"]
productos = [
    ("Lentes monofocales", 890, "Lentes"), ("Lentes bifocales", 1490, "Lentes"),
    ("Lentes progresivos", 2490, "Lentes"), ("Armazón básico", 350, "Armazones"),
    ("Armazón premium", 890, "Armazones"), ("Armazón de diseñador", 1650, "Armazones"),
    ("Solución 360ml", 120, "Accesorios"), ("Estuche rígido", 80, "Accesorios"),
    ("Microfibra", 45, "Accesorios"), ("Gotas lubricantes", 95, "Accesorios"),
    ("Consulta visual", 99, "Servicios"),
]

ciudades_map = {"Centro": "Culiacán", "Tres Ríos": "Culiacán", "Plaza Fiesta": "Culiacán",
    "Forum": "Culiacán", "Galerías": "Culiacán", "Mazatlán Centro": "Mazatlán",
    "Mazatlán Dorado": "Mazatlán", "Los Mochis": "Los Mochis", "Guasave": "Guasave"}

fecha_inicio = datetime(2025, 1, 1)
ventas = []
for i in range(500):
    fecha = fecha_inicio + timedelta(days=random.randint(0, 89))
    suc = random.choice(sucursales)
    prod, precio, cat = random.choice(productos)
    cant = random.randint(1, 3)
    desc = random.choice([0, 0, 0, 0, 0.05, 0.10, 0.15])
    total = round(precio * cant * (1 - desc), 2)
    ventas.append({"fecha": fecha.strftime("%Y-%m-%d"), "sucursal": suc, "ciudad": ciudades_map[suc],
        "producto": prod, "categoria": cat, "cantidad": cant, "precio_unitario": precio,
        "descuento": desc, "total": total})

ventas.sort(key=lambda x: x["fecha"])
with open("datos/ventas.csv", "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=ventas[0].keys())
    w.writeheader()
    w.writerows(ventas)

nombres = [("Ana","García"),("Luis","Hernández"),("María","López"),("Carlos","Martínez"),
    ("Sofía","Rodríguez"),("Pedro","Sánchez"),("Laura","Ramírez"),("Diego","Torres"),
    ("Valentina","Flores"),("Miguel","Díaz"),("Fernanda","Morales"),("Andrés","Jiménez"),
    ("Camila","Reyes"),("Roberto","Cruz"),("Isabella","Ortiz"),("Javier","Gutiérrez"),
    ("Daniela","Mendoza"),("Ricardo","Ruiz"),("Paulina","Alvarez"),("Emilio","Romero"),
    ("Andrea","Vargas"),("Tomás","Castillo"),("Lucía","Herrera"),("Gabriel","Medina"),
    ("Mariana","Castro"),("Sergio","Ríos"),("Natalia","Peña"),("Oscar","Aguilar"),
    ("Regina","Chávez"),("Alejandro","Navarro"),("Valeria","Guerrero"),("Francisco","Córdova"),
    ("Paula","Soto"),("Eduardo","Delgado"),("Renata","Molina"),("Héctor","Domínguez")]

puestos_sueldos = {"Gerente":(18000,25000),"Optometrista":(15000,22000),"Vendedor":(9000,14000),"Cajero":(8000,12000),"Asistente":(7000,10000)}
puestos = list(puestos_sueldos.keys())
empleados = []
for i,(n,a) in enumerate(nombres):
    suc = sucursales[i % len(sucursales)]
    puesto = random.choice(puestos)
    smin,smax = puestos_sueldos[puesto]
    empleados.append({"id":f"EMP{i+1:03d}","nombre":f"{n} {a}","puesto":puesto,
        "sucursal":suc,"ciudad":ciudades_map[suc],"sueldo":random.randint(smin,smax),
        "fecha_ingreso":(fecha_inicio - timedelta(days=random.randint(30,1500))).strftime("%Y-%m-%d")})

with open("datos/empleados.csv","w",newline="",encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=empleados[0].keys())
    w.writeheader()
    w.writerows(empleados)

print(f"✅ ventas.csv: {len(ventas)} registros (3 meses)")
print(f"✅ empleados.csv: {len(empleados)} registros")

---

## 1. ¿Qué es Pandas?

**Pandas** es la librería estándar de Python para análisis de datos. Su estructura principal es el **DataFrame** — una tabla con filas y columnas, similar a una hoja de Excel o una tabla de base de datos.

| Sin Pandas (notebook 06) | Con Pandas |
|--------------------------|------------|
| 10-20 líneas para leer y procesar CSV | 1 línea: `pd.read_csv()` |
| Loops manuales para filtrar | 1 línea: `df[df["columna"] > valor]` |
| Funciones propias para agrupar | 1 línea: `df.groupby("col").sum()` |
| Código largo para estadísticas | 1 línea: `df.describe()` |

### 1.1 Importar Pandas


In [ ]:
import pandas as pd  # pd es la convención universal

# Verificar versión
print(f"Pandas versión: {pd.__version__}")

### 1.2 Crear un DataFrame manualmente


In [ ]:
# Desde un diccionario (lo más común)
datos = {
    "sucursal": ["Centro", "Tres Ríos", "Plaza Fiesta", "Forum", "Galerías"],
    "ciudad": ["Culiacán"] * 5,
    "ventas": [185000, 143000, 220000, 157000, 192000],
    "empleados": [5, 4, 6, 4, 5],
}

df = pd.DataFrame(datos)
df  # En Colab/Jupyter se muestra como tabla bonita

In [ ]:
# Desde una lista de diccionarios
productos = [
    {"producto": "Lentes monofocales", "precio": 890, "stock": 45},
    {"producto": "Lentes bifocales", "precio": 1490, "stock": 30},
    {"producto": "Lentes progresivos", "precio": 2490, "stock": 20},
]

df_prod = pd.DataFrame(productos)
df_prod

---

## 2. Leer datos de archivos

Aquí es donde Pandas brilla — una línea para leer cualquier formato:


In [ ]:
# Leer CSV — una línea
df_ventas = pd.read_csv("datos/ventas.csv")

# ¿Qué tenemos?
print(f"Filas: {len(df_ventas)}")
print(f"Columnas: {list(df_ventas.columns)}")
print(f"Tipos de datos:\n{df_ventas.dtypes}")

In [ ]:
# Ver las primeras filas
df_ventas.head()

In [ ]:
# Ver las últimas filas
df_ventas.tail(3)

In [ ]:
# Leer empleados
df_empleados = pd.read_csv("datos/empleados.csv")
df_empleados.head()

In [ ]:
# Leer Excel (necesita openpyxl instalado)
# !pip install openpyxl -q  # Ya viene en Colab

# Leer una hoja específica
# df_inventario = pd.read_excel("datos/inventario_sucursales.xlsx", sheet_name="Inventario")
# df_inventario.head()

---

## 3. Explorar datos (EDA básico)

EDA = Exploratory Data Analysis. Lo primero que haces con cualquier dataset.

### 3.1 Información general


In [ ]:
# info() — resumen completo del DataFrame
df_ventas.info()

In [ ]:
# describe() — estadísticas de columnas numéricas
df_ventas.describe()

In [ ]:
# shape — (filas, columnas)
print(f"Dimensiones: {df_ventas.shape}")
print(f"Filas: {df_ventas.shape[0]}")
print(f"Columnas: {df_ventas.shape[1]}")

### 3.2 Valores únicos


In [ ]:
# ¿Cuántas sucursales hay?
print(f"Sucursales únicas: {df_ventas['sucursal'].nunique()}")
print(f"Lista: {df_ventas['sucursal'].unique()}")

# Conteo de valores
print(f"\nVentas por sucursal:")
print(df_ventas["sucursal"].value_counts())

In [ ]:
# Conteo por categoría
print("Ventas por categoría:")
print(df_ventas["categoria"].value_counts())

print(f"\nProductos únicos: {df_ventas['producto'].nunique()}")

### 3.3 Estadísticas rápidas de una columna


In [ ]:
# Estadísticas de la columna 'total'
print(f"Suma total:  ${df_ventas['total'].sum():,.0f}")
print(f"Promedio:    ${df_ventas['total'].mean():,.0f}")
print(f"Mediana:     ${df_ventas['total'].median():,.0f}")
print(f"Máximo:      ${df_ventas['total'].max():,.0f}")
print(f"Mínimo:      ${df_ventas['total'].min():,.0f}")
print(f"Desv. est.:  ${df_ventas['total'].std():,.0f}")

---

## 4. Seleccionar datos

### 4.1 Seleccionar columnas


In [ ]:
# Una columna — devuelve Series
totales = df_ventas["total"]
print(type(totales))
print(totales.head())

In [ ]:
# Múltiples columnas — devuelve DataFrame
seleccion = df_ventas[["fecha", "sucursal", "producto", "total"]]
seleccion.head()

### 4.2 Seleccionar filas por posición (iloc) y por etiqueta (loc)


In [ ]:
# iloc — por posición (como un array, empieza en 0)
print("Primera fila:")
print(df_ventas.iloc[0])

print("\nFilas 5 a 9:")
df_ventas.iloc[5:10]

In [ ]:
# iloc con filas Y columnas
# Primeras 5 filas, columnas 0 a 3
df_ventas.iloc[:5, :4]

In [ ]:
# loc — por etiqueta (nombre de columna)
df_ventas.loc[:4, ["fecha", "sucursal", "total"]]

---

## 5. Filtrar datos

Filtrar en Pandas es como un `WHERE` en SQL o un filtro en Excel, pero mucho más poderoso.

### 5.1 Filtros simples


In [ ]:
# Ventas mayores a $2,000
grandes = df_ventas[df_ventas["total"] > 2000]
print(f"Ventas > $2,000: {len(grandes)} de {len(df_ventas)}")
grandes.head()

In [ ]:
# Ventas de una sucursal específica
centro = df_ventas[df_ventas["sucursal"] == "Centro"]
print(f"Ventas de Centro: {len(centro)}")
print(f"Total Centro: ${centro['total'].sum():,.0f}")

### 5.2 Filtros con múltiples condiciones


In [ ]:
# AND — ambas condiciones
# Ventas en Culiacán con total > $1,000
filtro = df_ventas[(df_ventas["ciudad"] == "Culiacán") & (df_ventas["total"] > 1000)]
print(f"Ventas en Culiacán > $1,000: {len(filtro)}")
filtro.head()

In [ ]:
# OR — al menos una condición
# Ventas de lentes monofocales O progresivos
filtro = df_ventas[
    (df_ventas["producto"] == "Lentes monofocales") |
    (df_ventas["producto"] == "Lentes progresivos")
]
print(f"Ventas de monofocales o progresivos: {len(filtro)}")

In [ ]:
# isin() — buscar en una lista de valores
ciudades = ["Mazatlán", "Los Mochis"]
filtro = df_ventas[df_ventas["ciudad"].isin(ciudades)]
print(f"Ventas en {ciudades}: {len(filtro)}")
print(f"Total: ${filtro['total'].sum():,.0f}")

In [ ]:
# Filtro por texto con .str
# Productos que contienen "Lentes"
lentes = df_ventas[df_ventas["producto"].str.contains("Lentes")]
print(f"Ventas de lentes (cualquier tipo): {len(lentes)}")
print(f"Total: ${lentes['total'].sum():,.0f}")

In [ ]:
# Filtro por fechas
df_ventas["fecha"] = pd.to_datetime(df_ventas["fecha"])  # Convertir a fecha

# Ventas de enero
enero = df_ventas[df_ventas["fecha"].dt.month == 1]
print(f"Ventas en enero: {len(enero)}")
print(f"Total enero: ${enero['total'].sum():,.0f}")

# Ventas de la primera quincena de febrero
feb_q1 = df_ventas[
    (df_ventas["fecha"] >= "2025-02-01") &
    (df_ventas["fecha"] <= "2025-02-15")
]
print(f"\nVentas 1ra quincena febrero: {len(feb_q1)}")

---

## 6. Ordenar datos


In [ ]:
# Ordenar por total (mayor a menor)
top_ventas = df_ventas.sort_values("total", ascending=False)
top_ventas.head(10)

In [ ]:
# Ordenar por múltiples columnas
df_ventas.sort_values(["sucursal", "total"], ascending=[True, False]).head(10)

---

## 7. Crear y transformar columnas


In [ ]:
# Crear columna calculada
df_ventas["iva"] = (df_ventas["total"] * 0.16).round(2)
df_ventas["total_con_iva"] = (df_ventas["total"] * 1.16).round(2)

# Columna condicional con np.where
import numpy as np
df_ventas["ticket_size"] = np.where(
    df_ventas["total"] >= 2000, "Alto",
    np.where(df_ventas["total"] >= 500, "Medio", "Bajo")
)

df_ventas[["producto", "total", "iva", "total_con_iva", "ticket_size"]].head(10)

In [ ]:
# Columnas de fecha
df_ventas["mes"] = df_ventas["fecha"].dt.month
df_ventas["dia_semana"] = df_ventas["fecha"].dt.day_name()
df_ventas["semana"] = df_ventas["fecha"].dt.isocalendar().week.astype(int)

df_ventas[["fecha", "mes", "dia_semana", "semana"]].head(10)

In [ ]:
# apply() — aplicar una función personalizada
def clasificar_descuento(desc):
    if desc == 0:
        return "Sin descuento"
    elif desc <= 0.05:
        return "Descuento bajo"
    elif desc <= 0.10:
        return "Descuento medio"
    else:
        return "Descuento alto"

df_ventas["tipo_descuento"] = df_ventas["descuento"].apply(clasificar_descuento)
print(df_ventas["tipo_descuento"].value_counts())

---

## 8. Agrupar datos con groupby()

`groupby()` es el equivalente a las **tablas dinámicas** de Excel. Es una de las funciones más poderosas de Pandas.

### 8.1 Agrupar y sumar


In [ ]:
# Ventas totales por sucursal
ventas_suc = df_ventas.groupby("sucursal")["total"].sum().sort_values(ascending=False)
print("💰 Ventas totales por sucursal:")
print(ventas_suc.apply(lambda x: f"${x:,.0f}"))

In [ ]:
# Múltiples estadísticas a la vez con agg()
resumen = df_ventas.groupby("sucursal").agg(
    ventas_totales=("total", "sum"),
    transacciones=("total", "count"),
    ticket_promedio=("total", "mean"),
    venta_maxima=("total", "max"),
).sort_values("ventas_totales", ascending=False)

resumen["ticket_promedio"] = resumen["ticket_promedio"].round(0)
resumen

### 8.2 Agrupar por múltiples columnas


In [ ]:
# Ventas por ciudad y categoría
pivot = df_ventas.groupby(["ciudad", "categoria"])["total"].sum().unstack(fill_value=0)
pivot["Total"] = pivot.sum(axis=1)
pivot

In [ ]:
# Ventas por mes
ventas_mes = df_ventas.groupby("mes").agg(
    total=("total", "sum"),
    transacciones=("total", "count"),
    ticket_promedio=("total", "mean"),
)
ventas_mes["ticket_promedio"] = ventas_mes["ticket_promedio"].round(0)

meses = {1: "Enero", 2: "Febrero", 3: "Marzo"}
ventas_mes.index = ventas_mes.index.map(meses)
ventas_mes

### 8.3 Tablas dinámicas con pivot_table()


In [ ]:
# Tabla dinámica: ventas por sucursal y mes
tabla = pd.pivot_table(
    df_ventas,
    values="total",
    index="sucursal",
    columns="mes",
    aggfunc="sum",
    fill_value=0,
    margins=True,          # Agrega totales
    margins_name="Total"
)

# Renombrar columnas de mes
tabla.columns = [meses.get(c, c) for c in tabla.columns]
tabla

---

## 9. Análisis de empleados

Vamos a practicar con otro dataset:


In [ ]:
df_emp = pd.read_csv("datos/empleados.csv")
df_emp["fecha_ingreso"] = pd.to_datetime(df_emp["fecha_ingreso"])

# Columnas calculadas
df_emp["antiguedad_dias"] = (pd.Timestamp("2025-02-01") - df_emp["fecha_ingreso"]).dt.days
df_emp["antiguedad_anios"] = (df_emp["antiguedad_dias"] / 365).round(1)

df_emp.head()

In [ ]:
# Sueldo promedio por puesto
print("💼 Sueldo promedio por puesto:")
sueldo_puesto = df_emp.groupby("puesto")["sueldo"].agg(["mean", "min", "max", "count"])
sueldo_puesto.columns = ["Promedio", "Mínimo", "Máximo", "Empleados"]
sueldo_puesto = sueldo_puesto.sort_values("Promedio", ascending=False)
sueldo_puesto["Promedio"] = sueldo_puesto["Promedio"].round(0)
sueldo_puesto

In [ ]:
# Nómina por sucursal
nomina = df_emp.groupby("sucursal").agg(
    empleados=("id", "count"),
    nomina_mensual=("sueldo", "sum"),
    sueldo_promedio=("sueldo", "mean"),
    antiguedad_promedio=("antiguedad_anios", "mean"),
).sort_values("nomina_mensual", ascending=False)

nomina["sueldo_promedio"] = nomina["sueldo_promedio"].round(0)
nomina["antiguedad_promedio"] = nomina["antiguedad_promedio"].round(1)
nomina

---

## 10. Combinar DataFrames (merge)

Combinar tablas es como hacer un `JOIN` en SQL o un `VLOOKUP` en Excel:


In [ ]:
# Ventas por sucursal
ventas_suc = df_ventas.groupby("sucursal").agg(
    ventas_totales=("total", "sum"),
    transacciones=("total", "count"),
).reset_index()

# Nómina por sucursal
nomina_suc = df_emp.groupby("sucursal").agg(
    empleados=("id", "count"),
    nomina=("sueldo", "sum"),
).reset_index()

# Combinar (merge)
comparativa = pd.merge(ventas_suc, nomina_suc, on="sucursal", how="left")

# Calcular productividad
comparativa["venta_por_empleado"] = (comparativa["ventas_totales"] / comparativa["empleados"]).round(0)
comparativa["margen"] = comparativa["ventas_totales"] - comparativa["nomina"]

comparativa = comparativa.sort_values("venta_por_empleado", ascending=False)
comparativa

---

## 11. Exportar datos


In [ ]:
# Exportar a CSV
comparativa.to_csv("datos/comparativa_sucursales.csv", index=False)
print("✅ Exportado a CSV")

# Exportar a Excel con múltiples hojas
with pd.ExcelWriter("datos/analisis_completo.xlsx", engine="openpyxl") as writer:
    comparativa.to_excel(writer, sheet_name="Comparativa", index=False)
    ventas_mes.to_excel(writer, sheet_name="Ventas Mensuales")
    sueldo_puesto.to_excel(writer, sheet_name="Sueldos por Puesto")

print("✅ Exportado a Excel con 3 hojas")

In [ ]:
# Exportar solo un subset filtrado
lentes_culiacan = df_ventas[
    (df_ventas["categoria"] == "Lentes") &
    (df_ventas["ciudad"] == "Culiacán")
][["fecha", "sucursal", "producto", "cantidad", "total"]]

lentes_culiacan.to_csv("datos/lentes_culiacan.csv", index=False)
print(f"✅ lentes_culiacan.csv: {len(lentes_culiacan)} registros")

---

## 12. 🏆 Mini Proyecto: Dashboard de análisis completo

Vamos a generar un análisis completo usando todo lo aprendido:


In [ ]:
# 🏆 Mini Proyecto: Dashboard de Análisis

df = pd.read_csv("datos/ventas.csv")
df["fecha"] = pd.to_datetime(df["fecha"])
df["mes"] = df["fecha"].dt.month

print("=" * 65)
print("  📊 DASHBOARD DE VENTAS — VER DE VERDAD")
print("  Período: Enero - Marzo 2025")
print("=" * 65)

# --- KPIs Generales ---
print(f"\n  💰 Ventas Totales:     ${df['total'].sum():>12,.0f}")
print(f"  📦 Transacciones:      {len(df):>12,}")
print(f"  🎫 Ticket Promedio:    ${df['total'].mean():>12,.0f}")
print(f"  📈 Venta Máxima:       ${df['total'].max():>12,.0f}")
print(f"  📉 Venta Mínima:       ${df['total'].min():>12,.0f}")

# --- Tendencia mensual ---
print(f"\n{'─' * 65}")
print("  📅 TENDENCIA MENSUAL")
print(f"{'─' * 65}")

meses = {1: "Enero", 2: "Febrero", 3: "Marzo"}
for mes_num, mes_nombre in meses.items():
    mes_data = df[df["mes"] == mes_num]
    total = mes_data["total"].sum()
    trans = len(mes_data)
    barra = "█" * int(total / 10000)
    print(f"  {mes_nombre:<12} ${total:>10,.0f} ({trans} trans.) {barra}")

# --- Top sucursales ---
print(f"\n{'─' * 65}")
print("  🏆 TOP SUCURSALES")
print(f"{'─' * 65}")

top_suc = df.groupby("sucursal").agg(
    total=("total", "sum"),
    trans=("total", "count"),
).sort_values("total", ascending=False)

for i, (suc, row) in enumerate(top_suc.iterrows(), 1):
    ticket = row["total"] / row["trans"]
    pct = row["total"] / df["total"].sum() * 100
    medalla = ["🥇", "🥈", "🥉"][i-1] if i <= 3 else f" {i}."
    print(f"  {medalla} {suc:<20} ${row['total']:>10,.0f} ({pct:.1f}%) | Ticket: ${ticket:,.0f}")

# --- Top productos ---
print(f"\n{'─' * 65}")
print("  🛒 TOP 5 PRODUCTOS (por ingresos)")
print(f"{'─' * 65}")

top_prod = df.groupby("producto").agg(
    ingresos=("total", "sum"),
    unidades=("cantidad", "sum"),
).sort_values("ingresos", ascending=False).head(5)

for i, (prod, row) in enumerate(top_prod.iterrows(), 1):
    print(f"  {i}. {prod:<25} ${row['ingresos']:>10,.0f} ({row['unidades']} uds.)")

# --- Análisis de descuentos ---
print(f"\n{'─' * 65}")
print("  🏷️ ANÁLISIS DE DESCUENTOS")
print(f"{'─' * 65}")

con_desc = df[df["descuento"] > 0]
sin_desc = df[df["descuento"] == 0]
print(f"  Sin descuento: {len(sin_desc)} ventas (${sin_desc['total'].sum():,.0f})")
print(f"  Con descuento: {len(con_desc)} ventas (${con_desc['total'].sum():,.0f})")
print(f"  Descuento promedio: {con_desc['descuento'].mean():.1%}")

# --- Por ciudad ---
print(f"\n{'─' * 65}")
print("  📍 VENTAS POR CIUDAD")
print(f"{'─' * 65}")

por_ciudad = df.groupby("ciudad")["total"].agg(["sum", "count"]).sort_values("sum", ascending=False)
for ciudad, row in por_ciudad.iterrows():
    pct = row["sum"] / df["total"].sum() * 100
    print(f"  {ciudad:<18} ${row['sum']:>10,.0f} ({pct:.1f}%) — {int(row['count'])} transacciones")

print(f"\n{'=' * 65}")
print("  Generado con Python + Pandas | Culiacan.AI")
print(f"{'=' * 65}")

---

## 🔥 Retos

1. **Análisis de rendimiento:** Usando el merge de ventas + empleados, calcula qué sucursal tiene mejor ratio de ventas por empleado y genera un ranking. Identifica la sucursal más y menos eficiente.

2. **Tendencia semanal:** Agrupa las ventas por semana del año y encuentra cuál fue la mejor y peor semana. Genera un mini-gráfico de texto (barras con █) mostrando la tendencia.

3. **Recomendador de inventario:** Basado en las ventas de los últimos 3 meses, calcula cuántas unidades de cada producto se venden en promedio por mes. Usa eso para sugerir cuánto inventario tener para el próximo mes (con un factor de seguridad de 1.5x).


In [ ]:
# Reto 1: Análisis de rendimiento
# Tu código aquí 👇


In [ ]:
# Reto 2: Tendencia semanal
# Tu código aquí 👇


In [ ]:
# Reto 3: Recomendador de inventario
# Tu código aquí 👇


---

## 📋 Resumen

### Leer datos
| Operación | Código |
|-----------|--------|
| Leer CSV | `pd.read_csv("archivo.csv")` |
| Leer Excel | `pd.read_excel("archivo.xlsx", sheet_name="Hoja")` |

### Explorar
| Operación | Código |
|-----------|--------|
| Primeras filas | `df.head()` |
| Info general | `df.info()` |
| Estadísticas | `df.describe()` |
| Valores únicos | `df["col"].unique()`, `df["col"].nunique()` |
| Conteo | `df["col"].value_counts()` |

### Seleccionar y filtrar
| Operación | Código |
|-----------|--------|
| Una columna | `df["columna"]` |
| Varias columnas | `df[["col1", "col2"]]` |
| Filtrar | `df[df["col"] > valor]` |
| Múltiples filtros | `df[(cond1) & (cond2)]` |
| Buscar en lista | `df[df["col"].isin([...])` |
| Buscar texto | `df[df["col"].str.contains("texto")]` |

### Transformar
| Operación | Código |
|-----------|--------|
| Nueva columna | `df["nueva"] = df["col"] * 2` |
| Aplicar función | `df["col"].apply(funcion)` |
| Ordenar | `df.sort_values("col", ascending=False)` |

### Agrupar
| Operación | Código |
|-----------|--------|
| Agrupar y sumar | `df.groupby("col")["valor"].sum()` |
| Múltiples stats | `df.groupby("col").agg(...)` |
| Tabla dinámica | `pd.pivot_table(df, ...)` |

### Combinar y exportar
| Operación | Código |
|-----------|--------|
| Merge (JOIN) | `pd.merge(df1, df2, on="col")` |
| Exportar CSV | `df.to_csv("archivo.csv", index=False)` |
| Exportar Excel | `df.to_excel("archivo.xlsx", index=False)` |

---

## ⏭️ ¿Qué sigue?

En el siguiente notebook aprenderás **Visualización con Matplotlib** — cómo crear gráficas profesionales para comunicar tus análisis de datos.

👉 [08 — Visualización con Matplotlib](08_Visualizacion_con_Matplotlib.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
